In [56]:
import matplotlib.pyplot as plt # plotting
import csv


import numpy as np # linear algebra
from scipy.io import loadmat
import glob
import mne
import pandas as pd

### Setup data information

In [21]:
emotion_label = {'0':'neutral', '1':'sad', '2':'fear', '3':'happy'}
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
target_channels = [15, 23, 24, 32, 33, 41]
channel_mapping = ['FT7','FT8','T7','T8','TP7','TP8']

# location = "../SEED_IV/eeg_raw_data"


### Import files

In [22]:
file1 = glob.glob("../SEED_IV/eeg_raw_data/1/1_20160518.mat")
file2 = glob.glob("../SEED_IV/eeg_raw_data/1/2_20150915.mat")

# files = glob.glob(location+"/*/*.mat")
file1, file2

(['../SEED_IV/eeg_raw_data/1/1_20160518.mat'],
 ['../SEED_IV/eeg_raw_data/1/2_20150915.mat'])

In [23]:
subject1 = [loadmat(file1[0])]
subject2 = [loadmat(file2[0])]

In [95]:
ch_types = ['eeg'] * 6
sampling_freq = 1000

counter = {'0':1,'1':1,'2':1,'3':1}
for idx,s in enumerate(session1_label):
    # Naming
    count = counter[str(s)]
    counter[str(s)] =  counter[str(s)]+1
    label = emotion_label[str(s)]
    out_name = subject+label+'-'+str(count)+'.csv'
    # subject = 'subjecta-'
    # trial = 'cz_eeg'+str(idx+1)
    subject = 'subjectb-'
    trial = 'ha_eeg'+str(idx+1)
    #
    # trial = subject1[0][trial]
    trial = subject2[0][trial]
    channel = trial[target_channels[0]]
    channel2 = trial[target_channels[1]]
    channel3 = trial[target_channels[2]]
    channel4 = trial[target_channels[3]]
    channel5 = trial[target_channels[4]]
    channel6 = trial[target_channels[5]]
    raw_data=np.array([channel,channel2,channel3,channel4,channel5,channel6])

    info = mne.create_info(ch_names=channel_mapping,
                       ch_types=ch_types,
                       sfreq=sampling_freq)
    simulated_raw = mne.io.RawArray(raw_data, info)
    FFTed_signal = simulated_raw.filter(1, 70., fir_design='firwin')
    resampled_signal = FFTed_signal.resample(150, npad="auto")
    times = np.array([resampled_signal.times]).T

    # Write file
    header = ['timestamps', 'FT7','FT8','T7','T8','TP7','TP8']
    render_data = np.hstack((times,resampled_signal.get_data().T))
    render_data = np.vstack((header,render_data))
    np.savetxt('output/'+out_name, render_data, delimiter=",",fmt='%s')
    print(out_name)


Creating RawArray with float64 data, n_channels=6, n_times=33601
    Range : 0 ... 33600 =      0.000 ...    33.600 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 3301 samples (3.301 sec)

subjecta-sad-1.csv
Creating RawArray with float64 data, n_channels=6, n_times=19001
    Range : 0 ... 19000 =      0.000 ...    19.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-pha

In [39]:
subjects = [loadmat(file1[0])]
# for file in files:
data = np.array(subjects)
trial_no = 0
session1_label[trial_no]
trial = data[trial_no]['cz_eeg1']
channel = trial[target_channels[0]]
channel2 = trial[target_channels[1]]
channel3 = trial[target_channels[2]]
channel4 = trial[target_channels[3]]
channel5 = trial[target_channels[4]]
channel6 = trial[target_channels[5]]

raw_data=np.array([channel,channel2,channel3,channel4,channel5,channel6])
raw_data

array([[30.42817116, 28.10359001, 37.69993782, ..., 12.72559166,
        12.15934753, 13.41104507],
       [17.22574234, 14.7819519 , 28.13339233, ..., 13.14282417,
        17.1661377 , 18.53704453],
       [26.55386925, 30.42817116, 34.66010094, ..., 12.18914986,
        13.32163811, 11.20567322],
       [15.22898674, 16.39127731, 23.75245094, ..., 12.04013824,
        14.7819519 , 15.7058239 ],
       [21.10004425, 18.53704453, 25.1531601 , ..., 10.34140587,
         9.1791153 ,  7.56978989],
       [14.96076584, 16.59989357, 23.57363701, ..., 12.04013824,
        15.02037048, 14.27531242]])

### Create MNE object

In [40]:

ch_types = ['eeg'] * 6
sampling_freq = 1000

info = mne.create_info(channel_mapping, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')



<Info | 8 non-empty values
 bads: []
 ch_names: FT7, FT8, T7, T8, TP7, TP8
 chs: 6 EEG
 custom_ref_applied: False
 dig: 9 items (3 Cardinal, 6 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 6
 projs: []
 sfreq: 1000.0 Hz
>

In [43]:
info = mne.create_info(ch_names=channel_mapping,
                       ch_types=ch_types,
                       sfreq=sampling_freq)
simulated_raw = mne.io.RawArray(raw_data, info)
FFTed_signal = simulated_raw.filter(1, 70., fir_design='firwin')
resampled_signal = FFTed_signal.resample(150, npad="auto")  # set sampling frequency to 100Hz


Creating RawArray with float64 data, n_channels=6, n_times=33601
    Range : 0 ... 33600 =      0.000 ...    33.600 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 3301 samples (3.301 sec)



AttributeError: 'function' object has no attribute 'shape'

In [80]:
resampled_signal.get_data().T.shape
times = np.array([resampled_signal.times]).T
render_data = np.hstack((times,resampled_signal.get_data().T))

In [84]:
header = np.array(['timestamps', 'FT7','FT8','T7','T8','TP7','TP8'])

In [86]:
render_data = np.vstack((header,render_data))

In [87]:
render_data

array([['timestamps', 'FT7', 'FT8', ..., 'T8', 'TP7', 'TP8'],
       ['0.0', '0.3231370854592271', '1.133590591734884', ...,
        '1.189295652957236', '0.5499610486477706', '1.2128452186101504'],
       ['0.006666666666666667', '2.991708374268562',
        '12.227270271470074', ..., '15.223193582244246',
        '6.804479253325367', '15.693684775600781'],
       ...,
       ['33.58', '2.7537917746704745', '-2.5100183935135245', ...,
        '-3.525959525204028', '1.0350052908234424', '-2.040859203607187'],
       ['33.586666666666666', '1.087187789071921', '-7.8603025120764665',
        ..., '-7.136301883132551', '-2.3064059270331',
        '-5.509860694423277'],
       ['33.593333333333334', '-2.3552714209682137',
        '-5.247489403532302', ..., '-8.184155819819914',
        '-2.2755058789681004', '-6.481389602536014']], dtype='<U32')